<h1>Surrogate-Based Optimisation</h1>

In [2]:
from equadratures import *
import numpy as np

**Example 1: Constrained optimisation of Poly object with custom constraint functions**

We will demonstrate how to use the Optimisation class to solve the following optimisation problem

\begin{eqnarray}
    \min_{s_1,s_2} 	\quad    	& f(s_1, s_2) 	\\
    \textrm{ subject to } 	& (s_1-1)^3 - s_2 \leq 5 	\\
                            & s_1 + s_1 = 2 				\\
                            & -1 \leq s_1 \leq 1 			\\
                            & -1 \leq s_2 \leq 1.
\end{eqnarray}

where $f(s_1, s_2)$ is a Poly object and we will manually define the constraints. In this case, we will again use the 2D Styblinski-Tang function
$$f(\mathbf{s}) = \frac{1}{2} \sum_{i=1}^2 s_i^4 - 16s_i^2 + 5s_i$$
where each variable $s_1,s_2$ is uniformly distributed between $[-1,1]$. We will use 50 random samples to construct this fourth order Poly object.

In [3]:
from functions import StyblinskiTang

dims = 2
N = 50
poly_order = 4
S = np.random.uniform(-1,1,(N,dims))
y = evaluate_model(S, StyblinskiTang)

Defining the parameters and basis as before, we can compute the coefficients of the polynomial model via a Poly object.

In [4]:
my_params = [Parameter(poly_order, distribution='uniform',lower=-1.0, upper=1.0)\
 for _ in range(dims)]
my_basis = Basis('total-order')
my_poly = Poly(my_params, my_basis, method='least-squares', sampling_args={'sample-points':S,
                                                                          'sample-outputs':y})
my_poly.set_model()

Now that the polynomial surrogate model has been constructed for our objective, we can manually define our constraints.

In [5]:
def NonlinearConstraint(s):
    return 5.0 - (S[0]-1.0)**3 + S[1] 

Note that we write nonlinear constraints in the form $g(\mathbf{s}) \geq 0$ as this is the form required by Scipy.

In [6]:
Opt = Optimisation(method='trust-constr')
Opt.add_objective(poly=my_poly)
Opt.add_nonlinear_ineq_con(custom={'function': NonlinearConstraint})
Opt.add_bounds(-np.ones(dims), np.ones(dims))
Opt.add_linear_eq_con(np.array([1.0, 1.0]), 2.0)
sol = Opt.optimise(np.zeros(dims))
print("Calculated solution: optimal value of {} at {}".format(sol['fun'], sol['x']))

Calculated solution: optimal value of -9.99999991773416 at [1. 1.]


**Coding Task 1: Surrogate-based optimisation for normalised efficiency of a fan blade**
Using the test data provided (independent and identically uniformly distributed and bounded between -1 and 1), create a poly object of order 2 using the 'compressive-sensing' method and use the Optimisation class to maximise this poly object with the constraint that 
$$-\mathbf{1} \leq \mathbf{s} \leq \mathbf{1}$$ 
using the 'SLSQP' method.

HINT: To maximise a Poly object objective, simply write:

```python
Opt.add_objective(poly=my_poly, maximise=True)
```

In [6]:
S = np.loadtxt('bladeA_cs_training_inputs.dat')
f = np.loadtxt('bladeA_cs_training_outputs.dat')
m = S.shape[1]
####################################################################################
# WRITE YOUR CODE HERE
# Define Poly object with 'compressive-sensing' method
# Approximately 4 lines
my_params = [Parameter(distribution='uniform', lower=-1.0, upper=1.0, order=2) for i in range(m)]
my_basis = Basis('total-order')
my_poly = Poly(my_params, my_basis, method='compressive-sensing', sampling_args={'sample-points':S, 'sample-outputs':f})
my_poly.set_model()
# Initialise optimisation problem as 'Opt' using 'SLSQP' method
# Approximately 3 lines
Opt = Optimisation(method='SLSQP')
Opt.add_objective(poly=my_poly, maximise=True)
Opt.add_bounds(-np.ones(m),np.ones(m))
####################################################################################
sol = Opt.optimise(np.zeros(m))
print("Calculated solution: optimal value of {} at {}".format(sol['fun'], sol['x']))

Calculated solution: optimal value of 0.5600166314216701 at [ 1.          1.         -1.         -1.         -1.         -0.58863369
 -0.98285137 -1.         -1.         -1.          1.         -1.
  1.          1.          0.80602794 -1.         -1.          1.
  0.35424525  0.12706446 -1.          1.          1.          1.
  1.        ]


**Coding Task 2: Using Effective Quadratures to construct a trust-region method**
We will use Effective Quadratures and what we have learned about trust-region methods to construct our own simple trust-region method to solve the optimisation problem

\begin{equation}
\begin{split}
\min_{\mathbf{s}} \quad & f(\mathbf{s}) \\
\textrm{subject to} \quad & -\mathbf{1} \leq \mathbf{s} \leq \mathbf{1}.
\end{split}
\end{equation}

This task is broken up into the following small coding tasks:

A) Construct a quadratic model from given sample points $S$ and corresponding function evaluations $f$

B) Solve a trust-region subproblem to find a new potential iterate $\mathbf{s}_{k+1}$

**Coding Task 2A: Create quadratic models for trust-region method**

Given a set of points $S$, corresponding function evaluations $f$, and trust-region radius $\Delta_k$, create a function which creates a quadratic model using the 'least-squares' method, with parameters uniformly distributed and bounded within hypercube of radius $\Delta_k$ centred around the first point of $S$.

HINT: Points which are sampled within a hypercube of radius $r$ centred around $\hat{\mathbf{s}}$ have a lower bound of $\hat{s}_i - r$ and an upper bound of $\hat{s}_i + r$ for each coordinate.

In [7]:
%%writefile build_model.py
from equadratures import *

def build_model(S,f,del_k):
####################################################################################
    # WRITE YOUR CODE HERE 
    # Define Poly object with name 'my_poly' with 'least-squares' method
    # Approximately 3 lines
    my_params = Parameter(distribution='uniform', lower = S[0,i]-del_k, upper= S[0,i] + del_k)
####################################################################################
    my_poly.set_model()
    return my_poly

Overwriting build_model.py


**Coding Task 2B: Solving the trust-region subproblem**

Using the newly constructed model, current iterate $\mathbf{s}_k$ and trust-region radius $\Delta_k$, use the Optimisation class to solve the trust-region subproblem

\begin{equation}
\label{eq:subproblem}
\begin{split}
\min_{\mathbf{s}} \quad & m_k(\mathbf{s}) \\
\textrm{subject to} \quad & \| \mathbf{s} - \mathbf{s}_k \|_{\infty} \leq \Delta_k \\
& -\mathbf{1} \leq \mathbf{s} \leq \mathbf{1}
\end{split}
\end{equation}

using the 'SLSQP' method.

HINT: $\| \mathbf{A} \mathbf{x} - \mathbf{b} \|_{\infty} \leq 1$ is equivalent to $-\mathbf{1} \leq \mathbf{A} \mathbf{x} - \mathbf{b} \leq \mathbf{1}$

In [8]:
%%writefile compute_step.py
from equadratures import *
import numpy as np 

def compute_step(s_old,my_poly,del_k):
    Opt = Optimisation(method='SLSQP')
####################################################################################
    # WRITE YOUR CODE HERE 
    # Add objectives and constraints to the optimisation problem
    # Approximately 3 lines
    Opt.add_objective(poly=my_poly)
    Opt.add_bounds(-np.ones(s_old.size),np.ones(s_old.size))
    Opt.add_linear_ineq_con(np.eye(s_old.size), s_old-del_k*np.ones(s_old.size), s_old+del_k*np.ones(s_old.size))
####################################################################################
    sol = Opt.optimise(s_old)
    s_new = sol['x']
    m_new = sol['fun']
    return s_new, m_new

Overwriting compute_step.py


Now that we have written these functions, it is time to test out our trust-region method. Let us begin by again using the Styblinski-Tang function as our objective.

In [9]:
from trustregion import TrustRegion

TR = TrustRegion(StyblinskiTang)
s0 = np.zeros(dims)
sopt, fopt = TR.trust_region(s0)
print("Calculated solution: optimal value of {} at {}".format(fopt, sopt))

Calculated solution: optimal value of -19.999999999999993 at [-1. -1.]


Fantastic! Using Effective Quadratures, you have constructed a simple trust-region method to calculate local minima using localised quadratic models! However, how does this method compare to some other optimisation methods?

In [10]:
from compare import compare_optimisation
        
compare_optimisation(StyblinskiTang, s0)

Using our trust-region method, an optimal value of -19.999999999999993 was found after 21 function evaluations
Using COBYLA, an optimal value of -10.0 was found after 13 function evaluations
Using trust-constr, an optimal value of -19.991953540944284 was found after 21 function evaluations


We can see that our very simple trust-region method has relatively similar performance to a number of other common optimisation methods for the 2D Styblinski-Tang function. Let us test out this approach for the following:

- McCormick function $$f(s_1,s_2) = \sin(s_1 + s_2) + (s_1 + s_2)^2 - 1.5s_1 + 2.5s_2 +1$$

- Himmelblau function $$f(s_1,s_2) = (s_1^2 + s_2 - 11)^2 + (s_1 + s_2^2 - 7)^2$$

- Styblinski-Tang function $$f(\mathbf{s}) = \frac{1}{2} \sum_{i=1}^n (s_i^4 - 16s_i^2 + 5s_i)$$ for $n = 4, 8, 16$

In [6]:
from functions import Mccormick, Himmelblau
####################################################################################
# Feel free to change the function or the dims variable to match the problem. 
# Only StyblinskiTang can have dimensions more than 2.
dims = 2    
compare_optimisation(Mccormick, np.zeros(dims))
####################################################################################

Using our trust-region method, an optimal value of -1.528284046047533 was found after 36 function evaluations
Using COBYLA, an optimal value of -1.528284028086385 was found after 36 function evaluations
Using trust-constr, an optimal value of -1.5282827685834577 was found after 33 function evaluations
